In [1]:
import qutip as qt
import numpy as np
from math import cos, sin, acos, pow, sqrt
from cmath import exp
from itertools import *
pi = acos(0)*2

In [2]:
H = (cos(pi/8)*qt.basis(2,0) + sin(pi/8)*qt.basis(2,1)).unit()
plus = (qt.basis(2,0)+qt.basis(2,1)).unit()
X = qt.sigmax()
Y = qt.sigmay()
Z = qt.sigmaz()
I = qt.qeye(2)
S = Z.sqrtm()
T = S.sqrtm()
magic_states = [H, S*H, Z*H, S.dag()*H, X*H, S*X*H, Z*X*H, S.dag()*X*H, T*plus, S*T*plus, S.dag()*T.dag()*plus,T.dag()*plus]
magic_dms = [qt.ket2dm(s) for s in magic_states]
beta = acos(1/sqrt(3))/2
F = (cos(beta)*qt.basis(2,0)+exp(1j*pi/4)*sin(beta)*qt.basis(2,1)).unit()
face_states = [F, S*F, Z*F, S.dag()*F, X*F, S*X*F, S*Z*F, S.dag()*X*F]
face_dms = [qt.ket2dm(s) for s in face_states]

Using a result from [Aaronson & Gottesman](https://arxiv.org/abs/quant-ph/0406196) we can calculate the number of n-qubit pure stabiliser states

In [3]:
def n_stab(n):
    res = pow(2.,n)
    for i in range(n):
        res *= (pow(2.,n-i)+1)
    return res
def A(n):
    res = 1
    for i in range(n):
        res *= (pow(2.,n)-pow(2.,i))
    return res
def G(n):
    res = pow(2.,n)
    for i in range(n):
        res *= (pow(4.,n)/pow(2.,i) - pow(2.,i))
    return res
    
print(G(2))
print(A(2))
print(n_stab(2))

360.0
6.0
60.0


In [4]:
qt.Qobj(np.zeros((2,2)))
XX = qt.tensor(X, X)
ZZ = qt.tensor(Z,Z)
qt.commutator(XX,ZZ) == qt.Qobj(np.zeros((4,4)), dims=XX.dims)

True

In [6]:
base_elements = [I, X, Y, Z]#, -1*X, -1*Y, -1*Z]
strings = ['I', 'X', 'Y', 'Z']#, '-X', '-Y', '-Z']
elements = []
for pair in product(base_elements,base_elements):
    elements.append(pair)
elements = elements[1:] #Delete the II element
results = [qt.tensor(pair[0],pair[1]) for pair in elements]
res_strings = [pair[0]+pair[1] for pair in product(strings,strings)]
res_strings = res_strings[1:]
for n in range(15):
    results.append(-1*results[n])
    res_strings.append('-'+res_strings[n])


In [20]:
def test_equality(g1,g2):
    print(type(g1), type(g2))
    return all([_g1 in g2 for _g1 in g1])
        
_null = qt.Qobj(np.zeros([2,2]))
null = qt.tensor(_null,_null)

In [50]:
pairs = [] ##Build commuting pairs, as each group has 2 generators
for n, el in enumerate(results):
    for i in range(len(results)):
        if i == n:
            continue
        if -1*el==results[i]:
            continue
        if qt.commutator(el, results[i])== null:
            pairs.append((el, results[i]))
groups = []
for pair in pairs:
    candidate_group = [qt.tensor(I,I), pair[0], pair[1], pair[0]*pair[1]]
    if groups:
        if any([test_equality(candidate_group, group) for group in groups]):
               continue
        else:
               groups.append(candidate_group)
    else:
        groups.append(candidate_group)
len(groups)


60

In [8]:
def find_string(element):
    for n, mat in enumerate(results):
        if mat == element:
            return res_strings[n]
group_strings = []
for group in groups:
    group_strings.append([find_string(group[1]), find_string(group[2])])

In [36]:
def projector(group):
    I = group[0]
    res = ((I+group[1])/2) * ((I+group[2])/2)
    return res

In [53]:
states = []
for group in groups:
    eigs, vecs = projector(group).eigenstates(sort='high')
    for i in range(eigs.size):
        if np.allclose(eigs[i], complex(1)):
            states.append(vecs[i])
            break
len(states)


60

In [80]:
np.count_nonzero(SL0(A, b, 1e-12))

6

In [56]:
pairs = [pair for pair in combinations(states, 2)]
len(pairs)
_dims = [[2,2],[1,1]]
# self_products = set()
product_count = 0
self_product_count = 0
magic_count = 0
for pair in pairs:
    s1 = pair[0].unit()
    s2 = pair[1].unit()
    if qt.ket2dm(s1+s2) == null:
        continue
    dm = qt.ket2dm(1/sqrt(2)*(s1+s2))
    q1, q2 = dm.ptrace(0), dm.ptrace(1)
    if np.allclose(q1.tr(),1.):
        product_count +=1
        if q1 == q2:
            self_product_count += 1
#         self_products.add(tuple([q1.full().flatten()[i] for i in range(4)]))
            if q1 in magic_dms:
                magic_count += 1

print(product_count)
print(self_product_count)
print(magic_count)

718
89
4


In [ ]:
for el in self_products:
    dm = qt.Qobj([[el[0], el[1]],[el[2], el[3]]], dims=[[2],[2]])